In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Importing Libraries

In [ ]:
pip install split-folders

In [ ]:
# importing libraries
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, ZeroPadding2D
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import load_model
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import seaborn as sns

# Importing Data

In [ ]:
# path for main dataset
data = "../input/cell-images-for-detecting-malaria/cell_images/cell_images/"
os.listdir(data)

In [ ]:
# visualizing a test images
img = imread(data+"/Parasitized"+"/C99P60ThinF_IMG_20150918_141001_cell_93.png")

In [ ]:
# shape of the image
img.shape

In [ ]:
# setting image shape
image_shape = (130, 130, 3)

In [ ]:
# displaying an image.
plt.imshow(img)

# Pre-processing Data

In [ ]:
# setting a batch size
batch_size=32

In [ ]:
# splitting the data directory into training, validation and testing sets.
import splitfolders 
splitfolders.ratio(data, output="split_data", 
                   seed=42, ratio=(.8, .1, .1), 
                   group_prefix=None)

In [ ]:
# the paths for the data directorys
training_gen_path = './split_data/train'
testing_gen_path = './split_data/test'
validation_gen_path = './split_data/val'

# image preprocessing and setting up splited data for training the models.
image_gen = ImageDataGenerator()
bal_train_image_gen = image_gen.flow_from_directory(training_gen_path,
                                               target_size=(130, 130),
                                               color_mode="rgb",
                                               batch_size=batch_size,
                                               class_mode="binary",
                                               shuffle=False)

bal_test_image_gen = image_gen.flow_from_directory(validation_gen_path,
                                               target_size=(130, 130),
                                               color_mode="rgb",
                                               batch_size=batch_size,
                                               class_mode="binary",
                                              shuffle=False)

# Building Models

In [ ]:
# defining a callback earlystopping
# this helps us stop the training if the loss is increasing.
early_stop = EarlyStopping(monitor="val_loss", patience=2)

### Evaluation Function

In [ ]:
# defining the a class for evalution.
# this class is used to make prediction and return all the evaluation metrics like classification report
# ROC curve, Confusion matrix, amount of misclassified data. 
plt.rcParams["figure.figsize"] = (10,10)
class evaluate_model:
    def __init__(self, model, images, estimator_name):
        self.model = model
        self.images = images
        self.estimator_name = estimator_name
        
    # used for making predictions
    def create_preds(self):
        pred = self.model.predict(self.images)
        self.prediction = []
        for i in pred:
            if i > 0.5:
                self.prediction.append(1)
            else:
                self.prediction.append(0)

        return self.prediction
    
    # plotting roc curve
    def plot_roc_curve(self):
        y = self.images.labels
        pred = self.prediction
        fpr, tpr, thresholds = metrics.roc_curve(y, pred)
        roc_auc = metrics.auc(fpr, tpr)
        display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name=self.estimator_name)
        display.plot()
        plt.show()
    
    # building confusion matrix
    def get_cm(self):
        cm = confusion_matrix(self.images.labels, self.prediction)
        print(cm)
        fig, ax = plt.subplots(figsize=(12,12))
        sns.set(font_scale=1.6)
        sns.heatmap(cm, annot=True, linewidths=.5, ax=ax)
    
    # percent of misclassified data for each class
    def percent_incorrect_classification(self):
        cm = confusion_matrix(self.images.labels, self.prediction)
        incorr_fraction = 1 - np.diag(cm) / np.sum(cm, axis=1)
        fig, ax = plt.subplots(figsize=(12,12))
        plt.bar(np.arange(2), incorr_fraction)
        plt.xlabel('True Label')
        plt.ylabel('Fraction of incorrect predictions')
        plt.xticks(np.arange(2), self.images.class_indices)
        
    def evaluate(self):
        preds = self.create_preds()
        print('Accuracy of Model is: ',accuracy_score(self.images.labels, preds))
        
        print("\n")
        print("Classification Report")
        print(classification_report(self.images.labels, preds, target_names=self.images.class_indices))
        
        print("\n")
        print("ROC Curve")
        self.plot_roc_curve()
        
        print("\n")
        print("Confusion Matrix and Incorrect Classifications")
        self.get_cm()
        print("\n")
        self.percent_incorrect_classification()

In [ ]:
# setting up weights for each class in case there are any imbalances in the data.
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(bal_train_image_gen.classes),
                                        y = bal_train_image_gen.classes                                                    
                                    )
train_class_weights = dict(enumerate(class_weights))

### Optimization function

In [ ]:
# a grid containing losses and optimizers.
losses = ['binary_crossentropy', 'hinge']
optimizer = ['adam', 'SGD', 'RMSprop']

# function to run a grid search to get the best combination of loss function and optimizer.
def optimize(model):
    for i in losses:
        for j in optimizer:
            model.compile(loss=i, optimizer=j, metrics=["accuracy"])

            results = model.fit(bal_train_image_gen,
                                         epochs=10,
                                         validation_data=bal_test_image_gen,
                                         callbacks=[early_stop],
                                         verbose = 0, class_weight=train_class_weights)

            e = evaluate_model(model, test_image_gen, 'model 1')
            preds = e.create_preds()
            print('Loss = ' + i + ' ' + 'Optimizer = ' + j)
            print('Accuracy of Model is: ',accuracy_score(test_image_gen.labels, preds))

### Holdout testing data

In [ ]:
# setting up holdout data that is the data not seen by the model during training.
# used to evaluate the model performance.
bal_holdout_image_gen = image_gen.flow_from_directory(testing_gen_path,
                                               target_size=(130, 130),
                                               color_mode="rgb",
                                               batch_size=batch_size,
                                               class_mode="binary",
                                              shuffle=False)

## Defining and optimizing first model

In [ ]:
# the first model
# the base class is EfficientNet model.
base_model1 = tf.keras.applications.efficientnet.EfficientNetB7(input_shape=(130,130,3),include_top=False,weights="imagenet")
for layer in base_model1.layers:
    layer.trainable=False

# the base model is passed to a sequential model.
model1=Sequential()
model1.add(base_model1)
# dropout is implemented to avoid overfitting
model1.add(Dropout(0.2))
# the output is flattened
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(128, activation='relu'))
# the output of the model is a single neuron with sigmoid as activation function as this is a binary classification.
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

In [ ]:
# run grid search
optimize(model1)

## Optimized First Model

In [ ]:
# fit the model with best loss fucntion and optimizer.
base_model1 = tf.keras.applications.efficientnet.EfficientNetB7(input_shape=(130,130,3),include_top=False,weights="imagenet")
for layer in base_model1.layers:
    layer.trainable=False
    
model1=Sequential()
model1.add(base_model1)
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

model1.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# fit the model
results1 = model1.fit(train_image_gen,
                             epochs=10,
                             validation_data=test_image_gen,
                             callbacks=[early_stop])

# check the losses 
history1 = pd.DataFrame(model1.history.history)
history1[["loss", "val_loss"]].plot()

# save the model. 
model1.save('saved_models/model1.hdf5')

In [ ]:
# used to load the model 
# this can used to later if we dont want to train the model again. 
model1 = load_model('../input/final-ai-models/model2.hdf5')

### Evaluation first model

In [ ]:
# evaluate the model
e = evaluate_model(model1, bal_holdout_image_gen, 'model 1')
e.evaluate()

### Bagging using model 1

In [ ]:
# building the bagging ensemble. 
# save the models in the list
models = []
# save predictions. 
preds = []
# build three models. 
# the model is same as the first. 
for i in range(3):
    base_model1 = tf.keras.applications.efficientnet.EfficientNetB7(input_shape=(130,130,3),include_top=False,weights="imagenet")
    for layer in base_model1.layers:
        layer.trainable=False

    model=Sequential()
    model.add(base_model1)
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

    results1 = model.fit(bal_train_image_gen,
                                 epochs=5,
                                 validation_data=bal_test_image_gen,
                                 callbacks=[early_stop])
    e = evaluate_model(model, bal_test_image_gen, 'model')
    models.append(model)
    preds.append(e.create_preds())

In [ ]:
# save the three models. 
models[0].save('saved_models/m1.hdf5')
models[1].save('saved_models/m2.hdf5')
models[2].save('saved_models/m3.hdf5')

In [ ]:
# load models for future evaluations.
m1 = load_model('../input/ensemblemodels/m1.hdf5')
m2 = load_model('../input/ensemblemodels/m2.hdf5')
m3 = load_model('../input/ensemblemodels/m3.hdf5')
models = [m1, m2, m3]

In [ ]:
# averaging the predictions from the three models. 
preds = []
for m in models:
    e = evaluate_model(m, bal_holdout_image_gen, 'model')
    preds.append(e.create_preds())

preds=np.array(preds)
avg = np.mean(preds, axis=0, dtype=int)
print("Model 1 Bagging Ensemble Accuracy")
accuracy_score(bal_holdout_image_gen.labels, avg)

## Defining Second Model

In [ ]:
# define second model.
# vgg19 as base model
base_model2 = tf.keras.applications.vgg19.VGG19(input_shape=(130,130,3),include_top=False,weights="imagenet")
for layer in base_model2.layers:
    layer.trainable=False
    
model2=Sequential()
model2.add(base_model2)
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

In [ ]:
# run the grid search.
optimize(model2)

### Optimized 2nd Model

In [ ]:
# fit the model on optimal loss function and optimizer
base_model2 = tf.keras.applications.vgg19.VGG19(input_shape=(130,130,3),include_top=False,weights="imagenet")
for layer in base_model2.layers:
    layer.trainable=False
    
model2=Sequential()
model2.add(base_model2)
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

results2 = model2.fit(train_image_gen,
                             epochs=10,
                             validation_data=test_image_gen,
                             callbacks=[early_stop], class_weight=train_class_weights)

history2 = pd.DataFrame(model2.history.history)
history2[["loss", "val_loss"]].plot()

model2.save('saved_models/model2.hdf5')

In [ ]:
model2 = load_model('../input/final-ai-models/model2.hdf5')

### Evaluating Model 2

In [ ]:
# evaluate the model
e = evaluate_model(model2, bal_holdout_image_gen, 'model 2')
e.evaluate()

## Stacked Ensemble Model

In [ ]:
# load the models
model1 = load_model('../input/final-ai-models/model1.hdf5')
model2 = load_model('../input/final-ai-models/model2.hdf5')

In [ ]:
models = [model1, model2] #stacking individual models in a list
model_input = tf.keras.Input(shape=image_shape) #takes a list of tensors as input, all of the same shape
model_outputs = [model(model_input) for model in models] #collects outputs of models in a list
ensemble_output = tf.keras.layers.Average()(model_outputs) #averaging outputs
ensemble_model_final = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
ensemble_model_final.summary()

ensemble_model_final.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

ensemble_model_results_final = ensemble_model_final.fit(bal_train_image_gen,
                             epochs=10,
                             validation_data=bal_test_image_gen,
                             callbacks=[early_stop], class_weight=train_class_weights)

history2 = pd.DataFrame(ensemble_model_final.history.history)
history2[["loss", "val_loss"]].plot()

ensemble_model_final.save('saved_models/ensemble_model_final.hdf5')

In [ ]:
ensemble_model_final = load_model('../input/final-ai-models/ensemble_model_final.hdf5')

In [ ]:
# evaluate the model
e = evaluate_model(ensemble_model_final, bal_holdout_image_gen, 'ensemble of model 1 and 2')
e.evaluate()

## Testing On a Random Image

In [ ]:
# testing on random images.

# test_dir = "../input/cell-images-for-detecting-malaria/cell_images/Uninfected/C100P61ThinF_IMG_20150918_144104_cell_128.png"

test_dir = "../input/cell-images-for-detecting-malaria/cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_168.png"

In [ ]:
# checking the class assignment.
bal_holdout_image_gen.class_indices

In [ ]:
# a class to predict images
class test_image():
    def __init__(self, model, image_path):
        self.image_path = image_path
        self.image_shape = (130,130,3)
        self.model = model
        
    def predict_image(self):
        # load the images
        test_img = image.load_img(self.image_path, target_size=self.image_shape)
        # convert to array
        my_test_img_arr = image.img_to_array(test_img)
        # make it of right dimensions
        my_test_img_arr = np.expand_dims(my_test_img_arr, axis=0)
        # make prediction
        pred = self.model.predict(my_test_img_arr)

        prediction = None
        for i in pred:
            if i > 0.5:
                prediction = 1
            else:
                prediction = 0
        # return the predicted class
        return prediction

In [ ]:
img = imread("../input/cell-images-for-detecting-malaria/cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_168.png")
plt.imshow(img)

In [ ]:
# test the model on a random image
t = test_image(ensemble_model_final, test_dir)
t.predict_image()

In [ ]:
img = imread("../input/cell-images-for-detecting-malaria/cell_images/Uninfected/C100P61ThinF_IMG_20150918_144104_cell_128.png")
plt.imshow(img)

### Note: The code is my own implmentation including the flask implementation. 